In [4]:
pip install python-binance

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 3.9 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.1
    Uninstalling pytz-2024.1:
      Successfully uninstalled pytz-2024.1
Note: you may need to restart the kernel to use updated packages.


In [20]:
API_KEY = 'shdg4LieNnesTCcFqYwRwLHbXPRd8olrGEqsNjlOQTvxwoJ1hMPDmLAfyNBDjayn'
API_SECRET = 'lcL5A9efdtjVwFdRa0oSdf0q0FDKz8moUQmhyq5jycUdH4iApg9l2fh96Lduqx8l'

In [34]:
import logging
from binance.client import Client
from binance.enums import *
import sys

In [36]:
# Set up a friendly log file to keep track of everything
logging.basicConfig(
    filename='trading_bot.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


In [38]:
class BasicBot:
    def __init__(self, api_key, api_secret, testnet=True):
        self.client = Client(api_key, api_secret, testnet=testnet)
        self.base_url = "https://testnet.binancefuture.com"
        logging.info("Hey, bot is ready! Using Testnet URL: %s", self.base_url)
        print("Hey there! Your trading bot is all set up on the Binance Testnet. Let’s make some moves!")

    def validate_input(self, symbol, quantity, price=None, order_type=None, side=None):
        if not all([symbol, quantity, order_type, side]):
            raise ValueError("Oops! Looks like you forgot something—please fill in all the details!")
        if not isinstance(quantity, (int, float)) or quantity <= 0:
            raise ValueError("Hmm, quantity needs to be a positive number. Try again!")
        if order_type not in ["market", "limit", "stop_limit"]:
            raise ValueError("Whoa! Order type should be market, limit, or stop_limit. Pick one!")
        if side not in ["buy", "sell"]:
            raise ValueError("Hey, side should be buy or sell. Which one are you feeling?")
        if order_type == "limit" and not price:
            raise ValueError("For limit orders, I need a price to work with. Can you add that?")
        if order_type == "stop_limit" and not price:
            raise ValueError("Stop-limit needs a stop price. Let’s add that, okay?")
        return True

    def place_order(self, symbol, quantity, order_type, side, price=None, stop_price=None):
        try:
            self.validate_input(symbol, quantity, price, order_type, side)
            symbol = symbol.upper() + "USDT"

            if order_type == "market":
                print(f"Alright, placing a {side} market order for {quantity} {symbol}—here we go!")
                order = self.client.create_order(
                    symbol=symbol,
                    side=side.upper(),
                    type=ORDER_TYPE_MARKET,
                    quantity=quantity
                )
                logging.info("Just placed a %s market order for %s units of %s", side, quantity, symbol)

            elif order_type == "limit":
                print(f"Cool, setting a {side} limit order for {quantity} {symbol} at {price}! Let’s do this.")
                order = self.client.create_order(
                    symbol=symbol,
                    side=side.upper(),
                    type=ORDER_TYPE_LIMIT,
                    timeInForce=TIME_IN_FORCE_GTC,
                    quantity=quantity,
                    price=price
                )
                logging.info("Just placed a %s limit order for %s units of %s at %s", side, quantity, symbol, price)

            elif order_type == "stop_limit":
                print(f"Nice choice! Placing a {side} stop-limit order for {quantity} {symbol} at stop price {stop_price}.")
                order = self.client.create_order(
                    symbol=symbol,
                    side=side.upper(),
                    type=ORDER_TYPE_STOP_MARKET,
                    stopPrice=stop_price,
                    closePosition='true' if side == 'sell' else 'false',
                    quantity=quantity
                )
                logging.info("Just placed a %s stop_limit order for %s units of %s at stop price %s", side, quantity, symbol, stop_price)

            logging.info("Order response: %s", order)
            print(f"Success! Your {side} {order_type} order for {quantity} {symbol} is {order['status']}. Check the log for details!")
            return order

        except Exception as e:
            logging.error("Uh-oh, hit a snag: %s", str(e))
            print(f"Oops! Something went wrong: {str(e)}. Check the log for more info.")
            return None


In [40]:
def get_user_input():
    print("\nLet’s get started! Tell me what you want to trade.")
    symbol = input("What trading pair? (e.g., BTC): ").strip()
    quantity = float(input("How many units? (e.g., 0.1): ").strip())
    order_type = input("What kind of order? (market/limit/stop_limit): ").strip().lower()
    side = input("Buy or sell? (choose one): ").strip().lower()
    price = float(input("What price? (for limit/stop_limit, or 0 for market): ").strip()) if order_type in ["limit", "stop_limit"] else 0
    stop_price = float(input("What stop price? (for stop_limit, or 0 otherwise): ").strip()) if order_type == "stop_limit" else 0
    return symbol, quantity, order_type, side, price or None, stop_price or None


In [42]:
def main():
    api_key = "shdg4LieNnesTCcFqYwRwLHbXPRd8olrGEqsNjlOQTvxwoJ1hMPDmLAfyNBDjayn"
    api_secret = "lcL5A9efdtjVwFdRa0oSdf0q0FDKz8moUQmhyq5jycUdH4iApg9l2fh96Lduqx8l"

    if not api_key or not api_secret:
        print("Hey, I need your API key and secret to get started. Please set those up!")
        return

    bot = BasicBot(api_key, api_secret)
    
    while True:
        print("\nWhat would you like to do?")
        print("1. Place an Order")
        print("2. Call it a day")
        choice = input("Pick an option (1-2): ").strip()

        if choice == "1":
            symbol, quantity, order_type, side, price, stop_price = get_user_input()
            if order_type == "stop_limit" and stop_price == 0:
                print("Hold on! Stop-limit needs a stop price. Let’s try that again.")
                continue
            bot.place_order(symbol, quantity, order_type, side, price, stop_price)
        elif choice == "2":
            print("Alright, taking a break! See you next time!")
            break
        else:
            print("Hmm, that’s not an option. Let’s try 1 or 2!")

if __name__ == "__main__":
    main()

Hey there! Your trading bot is all set up on the Binance Testnet. Let’s make some moves!

What would you like to do?
1. Place an Order
2. Call it a day


Pick an option (1-2):  place an order


Hmm, that’s not an option. Let’s try 1 or 2!

What would you like to do?
1. Place an Order
2. Call it a day


Pick an option (1-2):  1



Let’s get started! Tell me what you want to trade.


What trading pair? (e.g., BTC):  BTS
How many units? (e.g., 0.1):  0.5
What kind of order? (market/limit/stop_limit):  market price
Buy or sell? (choose one):  sell


Oops! Something went wrong: Whoa! Order type should be market, limit, or stop_limit. Pick one!. Check the log for more info.

What would you like to do?
1. Place an Order
2. Call it a day


Pick an option (1-2):  2


Alright, taking a break! See you next time!
